In [1]:
!python -V

Python 3.9.16


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
import mlflow


mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

2023/04/20 12:42:04 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2023/04/20 12:42:04 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.


<Experiment: artifact_location='/home/ubuntu/mlops-zoomcamp2022/02-experiment-tracking/mlruns/1', creation_time=1681830312654, experiment_id='1', last_update_time=1681830312654, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [6]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [7]:
df_train = read_dataframe('./data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2021-02.parquet')

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [8]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [9]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [10]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [13]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715209766874

In [15]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [17]:
with mlflow.start_run():

    mlflow.set_tag("developer", "luca")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.parquet")

    alpha = 0.01
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(local_path="models/lin_reg.bin", artifact_path="models_pickle")

In [12]:
import xgboost as xgb

In [12]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [13]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [21]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [22]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

[15:40:33] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:7.86535                           
[1]	validation-rmse:6.86145                           
[2]	validation-rmse:6.73958                           
[3]	validation-rmse:6.71578                           
[4]	validation-rmse:6.70926                           
[5]	validation-rmse:6.70709                           
[6]	validation-rmse:6.70454                           
[7]	validation-rmse:6.70058                           
[8]	validation-rmse:6.69963                           
[9]	validation-rmse:6.69693                           
[10]	validation-rmse:6.69818                          
[11]	validation-rmse:6.69974                          
[12]	validation-rmse:6.70116                          
[13]	validation-rmse:6.70150                          
[14]	validation-rmse:6.70068                          
[15]	validation-rmse:6.70140                          
[

In [14]:
best_params = {
    'learning_rate': 0.283695451213546,
    'max_depth': 10,
    'min_child_weight': 1.4127402796335915,
    'objective': 'reg:linear',
    'reg_alpha': 0.013996218958663783,
    'reg_lambda': 0.0968439735905947,
    'seed': 42
}

mlflow.xgboost.autolog()

booster = xgb.train(
    params=best_params,
    dtrain=train,
    num_boost_round=1000,
    evals=[(valid, 'validation')],
    early_stopping_rounds=50
)

2023/04/19 14:00:29 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of xgboost. If you encounter errors during autologging, try upgrading / downgrading xgboost to a supported version, or try upgrading MLflow.
2023/04/19 14:00:29 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '86f46f2f3eaa44729fb006a0411fc011', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current xgboost workflow


[14:00:30] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.21418
[1]	validation-rmse:12.82225
[2]	validation-rmse:10.56430
[3]	validation-rmse:9.10153
[4]	validation-rmse:8.18284
[5]	validation-rmse:7.61384
[6]	validation-rmse:7.26368
[7]	validation-rmse:7.04469
[8]	validation-rmse:6.90540
[9]	validation-rmse:6.81526
[10]	validation-rmse:6.75513
[11]	validation-rmse:6.71234
[12]	validation-rmse:6.68312
[13]	validation-rmse:6.66555
[14]	validation-rmse:6.65062
[15]	validation-rmse:6.64196
[16]	validation-rmse:6.63615
[17]	validation-rmse:6.63023
[18]	validation-rmse:6.62408
[19]	validation-rmse:6.62002
[20]	validation-rmse:6.61607
[21]	validation-rmse:6.61237
[22]	validation-rmse:6.61039
[23]	validation-rmse:6.60750
[24]	validation-rmse:6.60480
[25]	validation-rmse:6.60249
[26]	validation-rmse:6.60117
[27]	validation-rmse:6.60029
[28]	validation-rmse:6.59905
[29]	validation-rmse:6.59729
[30]	validation-rm

2023/04/19 14:01:30 WARNING mlflow.xgboost: Failed to infer model signature: could not sample data to infer model signature: please ensure that autologging is enabled before constructing the dataset.
2023/04/19 14:01:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/.cache/pypoetry/virtualenvs/exp-tracking-Ew9w6R2h-py3.9/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


In [14]:
mlflow.xgboost.autolog(disable=True)

with mlflow.start_run():
    
    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.283695451213546,
        'max_depth': 10,
        'min_child_weight': 1.4127402796335915,
        'objective': 'reg:linear',
        'reg_alpha': 0.013996218958663783,
        'reg_lambda': 0.0968439735905947,
        'seed': 42
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[12:55:13] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:16.21418
[1]	validation-rmse:12.82225
[2]	validation-rmse:10.56430
[3]	validation-rmse:9.10153
[4]	validation-rmse:8.18284
[5]	validation-rmse:7.61384
[6]	validation-rmse:7.26368
[7]	validation-rmse:7.04469
[8]	validation-rmse:6.90540
[9]	validation-rmse:6.81526
[10]	validation-rmse:6.75513
[11]	validation-rmse:6.71234
[12]	validation-rmse:6.68312
[13]	validation-rmse:6.66555
[14]	validation-rmse:6.65062
[15]	validation-rmse:6.64196
[16]	validation-rmse:6.63615
[17]	validation-rmse:6.63023
[18]	validation-rmse:6.62408
[19]	validation-rmse:6.62002
[20]	validation-rmse:6.61607
[21]	validation-rmse:6.61237
[22]	validation-rmse:6.61039
[23]	validation-rmse:6.60750
[24]	validation-rmse:6.60480
[25]	validation-rmse:6.60249
[26]	validation-rmse:6.60117
[27]	validation-rmse:6.60029
[28]	validation-rmse:6.59905
[29]	validation-rmse:6.59729
[30]	validation-rm

/home/ubuntu/.cache/pypoetry/virtualenvs/exp-tracking-Ew9w6R2h-py3.9/lib/python3.9/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [15]:
logged_model = 'runs:/8e4cc14ed4a54858a903f6d95168b7d2/models_mlflow'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

[13:24:59] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: 8e4cc14ed4a54858a903f6d95168b7d2

In [16]:
xgboost_model = mlflow.xgboost.load_model(logged_model)

[13:27:14] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.


In [17]:
xgboost_model

In [18]:
y_pred = xgboost_model.predict(valid)

In [19]:
y_pred[:10]

array([14.824908,  7.24244 , 14.117635, 24.483923,  9.366333, 17.152206,
       11.729501,  9.136774,  8.904758, 19.355263], dtype=float32)

In [20]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

mlflow.sklearn.autolog()

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):

    with mlflow.start_run():

        mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
        mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")
        mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)
        

/home/ubuntu/.cache/pypoetry/virtualenvs/exp-tracking-Ew9w6R2h-py3.9/lib/python3.9/site-packages/sklearn/svm/_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
